In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import theano.tensor as tt
import seaborn as sns
import matplotlib as mpl
from ipywidgets import interactive

In [2]:
from platform import python_version
print('Running on pyMC3 v{}'.format(pm.__version__))
print('Running Python version v{}'.format(python_version()))

Running on pyMC3 v3.7
Running Python version v3.7.4


Model:   
Underlying log rate follows Normal distribution.
$$z_{t} \sim N(\mu, \sigma^{2})$$  

Overdose rate is deterministic variable that exponentiates log rate.
$$\lambda_{t}^{OD} = \exp(z_{t})$$

Overdoses follows Poission distribution.
$$O_{t} \sim Poi(\lambda_{t}^{OD}N)$$  
where t= 1,2,3, ..., 12.  
I let $\mu$=-4, $\sigma$=1.


In [4]:
import random
random.seed(1)
N=10000
z_t = np.random.normal(loc=-4, scale=1, size = 12)
o_t = np.random.poisson(lam=(np.exp(z_t)*N))
print('Vector z_t: ', z_t)
print('Vector o_t: ', o_t)

Vector z_t:  [-4.35769363 -4.24549652 -1.92966025 -4.34330515 -2.61057079 -2.72056553
 -3.5191374  -4.51571647 -4.56809845 -4.28470053 -3.65684721 -3.4595028 ]
Vector o_t:  [ 127  152 1384  132  731  704  300  121  112  141  256  299]


Now we have our vector of observations.  
Before fitting a model to the data, let's assume that $p(\mu)$ and $p(\sigma^2)$ is noninformative, say uniform.
$$p(\mu) = U(-10, 10)$$  
$$p(\sigma) = U(-3, 3)$$  
Let's fit the model to the data o_t.

In [5]:
with pm.Model() as model:
    mu = pm.Uniform('mu', lower = -10, upper= 0)
    sigma = pm.Uniform('sigma', lower = 3, upper= 3)
    z_t =pm.Normal('z_t',mu=mu, sigma=sigma, shape = (12,))
    
    x = pm.Poisson('o_t', mu=tt.exp(z_t)*N, shape=(12,), observed=o_t)

In [6]:
with model:
    trace = pm.sample(1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [z_t, sigma, mu]
Sampling 2 chains:   0%|          | 0/3000 [00:00<?, ?draws/s]/Users/hyeongcheolpark/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
Sampling 2 chains:   0%|          | 0/3000 [00:02<?, ?draws/s]
Bad initial energy, check any log probabilities that are inf or -inf, nan or very small:
sigma_interval__   NaN


ParallelSamplingError: Bad initial energy

/Users/hyeongcheolpark/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
